In [1]:
from google.colab import drive
drive.mount('/content/gdrive')

Mounted at /content/gdrive


In [2]:
cd /content/gdrive/MyDrive/horse detection and segmentation

/content/gdrive/MyDrive/horse detection and segmentation


In [3]:
cd yolov5

/content/gdrive/MyDrive/horse detection and segmentation/yolov5


In [4]:
!pip install -r requirements.txt  # install

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 184.3/184.3 kB 7.7 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 586.5/586.5 kB 52.3 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 62.7/62.7 kB 6.7 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 205.1/205.1 kB 21.2 MB/s eta 0:00:00


In [5]:
cd ..

/content/gdrive/MyDrive/horse detection and segmentation


In [6]:
!git clone https://github.com/facebookresearch/segment-anything.git

fatal: destination path 'segment-anything' already exists and is not an empty directory.


In [7]:
cd segment-anything

/content/gdrive/MyDrive/horse detection and segmentation/segment-anything


In [8]:
!pip install -e .

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
Obtaining file:///content/gdrive/MyDrive/horse%20detection%20and%20segmentation/segment-anything
  Preparing metadata (setup.py) ... done
  Running setup.py develop for segment-anything


In [ ]:
!{sys.executable} -m pip install 'git+https://github.com/facebookresearch/segment-anything.git'

/bin/bash: {sys.executable}: command not found


In [10]:
cd ..

/content/gdrive/MyDrive/horse detection and segmentation


In [11]:
from segment_anything import sam_model_registry, SamAutomaticMaskGenerator, SamPredictor


In [12]:
!pip install supervision

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 42.1/42.1 kB 2.9 MB/s eta 0:00:00


In [13]:
import numpy as np
import torch
import matplotlib.pyplot as plt
import cv2

In [14]:
# Import required libraries
import torch
from PIL import Image
from pathlib import Path
from IPython.display import Image as display_image

# Load pre-trained YOLOv5 model
model = torch.hub.load('ultralytics/yolov5', 'yolov5s', pretrained=True)
image_dir = Path('images/')
boxes=[]

for image_path in image_dir.glob('*.jpg'):

    image = Image.open(image_path)
    results = model(image)
    
    # [x_min, y_min, x_max, y_max, confidence, class].
    bboxes = results.xyxy[0].tolist()

    for bbox in bboxes:
      print(bbox)
      if bbox[5] == 17.0:
        boxes.append(bbox[:4])
    
    # Display the image with bounding boxes around detected objects
    results.show()

In [ ]:
print(boxes)

[[818.638916015625, 345.2298278808594, 1434.9404296875, 789.0404052734375], [370.6645202636719, 360.2322692871094, 1271.0662841796875, 894.696533203125], [738.1558837890625, 333.25048828125, 1559.46240234375, 877.360107421875], [774.31298828125, 344.09368896484375, 1724.183349609375, 873.4612426757812], [535.3876953125, 369.6917724609375, 1306.2186279296875, 872.27978515625], [465.0296630859375, 334.25201416015625, 1207.08642578125, 929.2989501953125]]


In [ ]:
import sys
sys.path.append("..")
from segment_anything import sam_model_registry, SamPredictor

sam_checkpoint = "sam_vit_h_4b8939.pth"
model_type = "vit_h"

device = "cuda"

sam = sam_model_registry[model_type](checkpoint=sam_checkpoint)
sam.to(device=device)

predictor = SamPredictor(sam)
mask_predictor = SamPredictor(sam)

In [ ]:
import numpy as np
import cv2
import supervision as sv

for i in range(6):
  box = boxes[i]
  box = np.array(box)
  image_bgr = cv2.imread(f"images/input{i+1}.jpg")
  image_rgb = cv2.cvtColor(image_bgr, cv2.COLOR_BGR2RGB)
  mask_predictor.set_image(image_rgb)
  masks, scores, logits = mask_predictor.predict(
      box=box,
      multimask_output=True
      )
  box_annotator = sv.BoxAnnotator(color=sv.Color.red())
  mask_annotator = sv.MaskAnnotator(color=sv.Color.red())
  detections = sv.Detections(
      xyxy=sv.mask_to_xyxy(masks=masks),
      mask=masks
      )
  detections = detections[detections.area == np.max(detections.area)]
  source_image = box_annotator.annotate(scene=image_bgr.copy(), detections=detections, skip_label=True)
  segmented_image = mask_annotator.annotate(scene=image_bgr.copy(), detections=detections)
  
  cv2.imwrite(f"output{i+1}.jpg", segmented_image)

  sv.plot_images_grid(
      images=[source_image, segmented_image],
      grid_size=(1, 2),
      titles=['source image', 'segmented image']
      )
  